In [0]:
!git clone "https://github.com/convman/Multimodal-MOSEI.git"


In [2]:
cd Multimodal-MOSEI/

/content/Multimodal-MOSEI


In [0]:
!chmod +x data/dataset_download.sh
!./data/dataset_download.sh

In [4]:
import pandas as pd
import numpy as np
import h5py

Test_labels_fear = pd.read_csv("mosi2uni_Test_labels_fear.csv",header=None)
Train_labels_fear = pd.read_csv("mosi2uni_Train_labels_fear.csv",header=None)

video_train = h5py.File("video_train.h5","r")
print(list(video_train.keys()))
video_train = np.array(video_train.get('d1'))

video_test = h5py.File("video_test.h5","r")
print(list(video_test.keys()))
video_test = np.array(video_test.get('d1'))

audio_train = h5py.File("audio_train.h5","r")
print(list(audio_train.keys()))
audio_train = np.array(audio_train.get('d1'))

audio_test = h5py.File("audio_test.h5","r")
print(list(audio_test.keys()))
audio_test = np.array(audio_test.get('d1'))

text_train_emb = h5py.File("text_train_emb.h5","r")
print(list(text_train_emb.keys()))
text_train_emb = np.array(text_train_emb.get('d1'))

text_test_emb = h5py.File("text_test_emb.h5","r")
print(list(text_test_emb.keys()))
text_test_emb = np.array(text_test_emb.get('d1'))
text_test_emb.shape


['d1']
['d1']
['d1']
['d1']
['d1']
['d1']


(4832, 20, 300)

In [5]:

from keras.models import load_model
import keras
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import tensorflow as tf
from time import time
from keras import layers
from google.colab import files  
from keras.models import load_model
from keras.models import Model,Sequential,Model
from keras.layers import *
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from keras import callbacks
from keras.callbacks import *
%matplotlib inline  

Using TensorFlow backend.


In [6]:
model_audio_fear = load_model("unimodal baselines/weights/audio/weights_rnn_fear.h5")
model_video_fear = load_model("unimodal baselines/weights/video/weights_rnn_fear.h5")
model_text_fear = load_model("unimodal baselines/weights/text/weights_rnn_fear.h5")
early_fusion_fear = load_model("multimodal baselines/v2 - early fusion/weights/early_fusion_fear.h5")


W0703 11:07:02.901034 140246129813376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0703 11:07:02.947749 140246129813376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0703 11:07:02.952060 140246129813376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0703 11:07:05.196143 140246129813376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0703 11:07:05.197472 1402461298

In [0]:

concat_test = np.concatenate((video_test,audio_test),axis=-1)
concat_test = np.concatenate((concat_test, text_test_emb),axis=-1)
concat_train = np.concatenate((video_train,audio_train),axis=-1)
concat_train = np.concatenate((concat_train, text_train_emb),axis=-1)


In [0]:

for layer in model_audio_fear.layers[:-1]:   
    layer.trainable = False
for layer in model_text_fear.layers[:-1]:   
    layer.trainable = False
for layer in model_video_fear.layers[:-1]:   
    layer.trainable = False
for layer in early_fusion_fear.layers[:-1]:   
    layer.trainable = False


In [0]:
m_ef = Sequential()
i=0
for layer in early_fusion_fear.layers[:]:
  layer.name = str(i)
  i=i+1
  m_ef.add(layer)

In [14]:
i_a = Input(shape=(20,74))
i_t = Input(shape=(20,300))
i_v = Input(shape=(20,35))
i_ef = Input(shape=(20,409))
y = concatenate([model_audio_fear(i_a),model_text_fear(i_t),model_video_fear(i_v),m_ef(i_ef)])
y = Dense(1,activation='sigmoid')(y)
model = Model([i_a,i_t,i_v,i_ef],y)
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 20, 74)       0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 20, 300)      0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, 20, 35)       0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 20, 409)      0                                            
__________________________________________________________________________________________________
model_3 (M

In [15]:
model.compile('adam','binary_crossentropy',metrics=['accuracy'],)
es = EarlyStopping(monitor='val_acc',mode='max' ,patience=5, min_delta=0.0001,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,patience=2,min_lr=0.00001)
model.fit([audio_train,text_train_emb,video_train,concat_train],Train_labels_fear,128,epochs=20,validation_split=0.1,callbacks=[es,reduce_lr])
print(model.evaluate([audio_test,text_test_emb,video_test,concat_test],Test_labels_fear))


Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 70s 5ms/step - loss: 0.6711 - acc: 0.6132 - val_loss: 0.6033 - val_acc: 0.8875
Epoch 2/20
13761/13761 [==============================] - 58s 4ms/step - loss: 0.5005 - acc: 0.9345 - val_loss: 0.4776 - val_acc: 0.9117
Epoch 3/20
13761/13761 [==============================] - 58s 4ms/step - loss: 0.4148 - acc: 0.9237 - val_loss: 0.4240 - val_acc: 0.9111
Epoch 4/20
13761/13761 [==============================] - 58s 4ms/step - loss: 0.3650 - acc: 0.9214 - val_loss: 0.3893 - val_acc: 0.9111
Epoch 5/20
13761/13761 [==============================] - 60s 4ms/step - loss: 0.3414 - acc: 0.9208 - val_loss: 0.3836 - val_acc: 0.9111
Epoch 6/20
13761/13761 [==============================] - 60s 4ms/step - loss: 0.3345 - acc: 0.9209 - val_loss: 0.3781 - val_acc: 0.9111
Epoch 7/20
13761/13761 [==============================] - 59s 4ms/step - loss: 0.3303 - acc: 0.9206 - val_loss: 0.3770 - val_acc:

In [0]:

model.save("ensamble_fear.h5")
files.download("ensamble_fear.h5")

In [0]:
Test_labels_angry = pd.read_csv("mosi2uni_Test_labels_angry.csv",header=None)
Train_labels_angry = pd.read_csv("mosi2uni_Train_labels_angry.csv",header=None)

In [0]:
model_audio_angry = load_model("unimodal baselines/weights/audio/weights_rnn_angry.h5")
model_video_angry = load_model("unimodal baselines/weights/video/weights_rnn_angry.h5")
model_text_angry = load_model("unimodal baselines/weights/text/weights_rnn_angry.h5")
early_fusion_angry = load_model("multimodal baselines/v2 - early fusion/weights/early_fusion_angry.h5")


In [0]:
concat_test = np.concatenate((video_test,audio_test),axis=-1)
concat_test = np.concatenate((concat_test, text_test_emb),axis=-1)
concat_train = np.concatenate((video_train,audio_train),axis=-1)
concat_train = np.concatenate((concat_train, text_train_emb),axis=-1)

In [0]:

for layer in model_audio_angry.layers[:-1]:   
    layer.trainable = False
for layer in model_text_angry.layers[:-1]:   
    layer.trainable = False
for layer in model_video_angry.layers[:-1]:   
    layer.trainable = False
for layer in early_fusion_angry.layers[:-1]:   
    layer.trainable = False

In [0]:

m_ef = Sequential()
i=0
for layer in early_fusion_angry.layers[:]:
    layer.name = str(i)
    i=i+1
    m_ef.add(layer)

In [22]:

i_a = Input(shape=(20,74))
i_t = Input(shape=(20,300))
i_v = Input(shape=(20,35))
i_ef = Input(shape=(20,409))
y = concatenate([model_audio_angry(i_a),model_text_angry(i_t),model_video_angry(i_v),m_ef(i_ef)])
y = Dense(1,activation='sigmoid')(y)
model = Model([i_a,i_t,i_v,i_ef],y)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 20, 74)       0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 20, 300)      0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           (None, 20, 35)       0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 20, 409)      0                                            
__________________________________________________________________________________________________
model_2 (M

In [23]:
model.compile('adam','binary_crossentropy',metrics=['accuracy'],)
es = EarlyStopping(monitor='val_acc',mode='max' ,patience=5, min_delta=0.0001,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,patience=2,min_lr=0.00001)
model.fit([audio_train,text_train_emb,video_train,concat_train],Train_labels_angry,128,epochs=20,validation_split=0.1,callbacks=[es,reduce_lr])
print(model.evaluate([audio_test,text_test_emb,video_test,concat_test],Test_labels_angry))


Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 77s 6ms/step - loss: 0.6269 - acc: 0.7551 - val_loss: 0.6127 - val_acc: 0.7646
Epoch 2/20
13761/13761 [==============================] - 59s 4ms/step - loss: 0.5313 - acc: 0.8701 - val_loss: 0.5386 - val_acc: 0.7999
Epoch 3/20
13761/13761 [==============================] - 59s 4ms/step - loss: 0.4785 - acc: 0.8479 - val_loss: 0.5033 - val_acc: 0.8110
Epoch 4/20
13761/13761 [==============================] - 59s 4ms/step - loss: 0.4492 - acc: 0.8373 - val_loss: 0.4873 - val_acc: 0.8090
Epoch 5/20
13761/13761 [==============================] - 59s 4ms/step - loss: 0.4302 - acc: 0.8365 - val_loss: 0.4798 - val_acc: 0.8090
Epoch 6/20
13761/13761 [==============================] - 60s 4ms/step - loss: 0.4209 - acc: 0.8443 - val_loss: 0.4789 - val_acc: 0.8071
Epoch 7/20
13761/13761 [==============================] - 60s 4ms/step - loss: 0.4180 - acc: 0.8459 - val_loss: 0.4779 - val_acc:

In [0]:
model.save("ensamble_angry.h5")
files.download("ensamble_angry.h5")

In [0]:

Test_labels_disgust = pd.read_csv("mosi2uni_Test_labels_disgust.csv",header=None)
Train_labels_disgust = pd.read_csv("mosi2uni_Train_labels_disgust.csv",header=None)


In [0]:
model_audio_disgust = load_model("unimodal baselines/weights/audio/weights_rnn_disgust.h5")
model_video_disgust = load_model("unimodal baselines/weights/video/weights_rnn_disgust.h5")
model_text_disgust = load_model("unimodal baselines/weights/text/weights_rnn_disgust.h5")
early_fusion_disgust = load_model("multimodal baselines/v2 - early fusion/weights/early_fusion_disgust.h5")


In [0]:
concat_test = np.concatenate((video_test,audio_test),axis=-1)
concat_test = np.concatenate((concat_test, text_test_emb),axis=-1)
concat_train = np.concatenate((video_train,audio_train),axis=-1)
concat_train = np.concatenate((concat_train, text_train_emb),axis=-1)


In [0]:
for layer in model_audio_disgust.layers[:-1]:   
    layer.trainable = False
for layer in model_text_disgust.layers[:-1]:   
    layer.trainable = False
for layer in model_video_disgust.layers[:-1]:   
    layer.trainable = False
for layer in early_fusion_disgust.layers[:-1]:   
    layer.trainable = False


In [0]:
m_ef = Sequential()
i=0
for layer in early_fusion_disgust.layers[:]:
    layer.name = str(i)
    i=i+1
    m_ef.add(layer)

In [30]:
i_a = Input(shape=(20,74))
i_t = Input(shape=(20,300))
i_v = Input(shape=(20,35))
i_ef = Input(shape=(20,409))
y = concatenate([model_audio_disgust(i_a),model_text_disgust(i_t),model_video_disgust(i_v),m_ef(i_ef)])
y = Dense(1,activation='sigmoid')(y)
model = Model([i_a,i_t,i_v,i_ef],y)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 20, 74)       0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           (None, 20, 300)      0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           (None, 20, 35)       0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           (None, 20, 409)      0                                            
__________________________________________________________________________________________________
model_5 (M

In [31]:
model.compile('adam','binary_crossentropy',metrics=['accuracy'],)
es = EarlyStopping(monitor='val_acc',mode='max' ,patience=5, min_delta=0.0001,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,patience=2,min_lr=0.00001)
model.fit([audio_train,text_train_emb,video_train,concat_train],Train_labels_disgust,128,epochs=20,validation_split=0.1,callbacks=[es,reduce_lr])
print(model.evaluate([audio_test,text_test_emb,video_test,concat_test],Test_labels_disgust))

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 84s 6ms/step - loss: 0.6952 - acc: 0.7849 - val_loss: 0.5684 - val_acc: 0.8489
Epoch 2/20
13761/13761 [==============================] - 60s 4ms/step - loss: 0.5487 - acc: 0.8191 - val_loss: 0.4340 - val_acc: 0.8489
Epoch 3/20
13761/13761 [==============================] - 59s 4ms/step - loss: 0.4215 - acc: 0.8191 - val_loss: 0.3989 - val_acc: 0.8489
Epoch 4/20
13761/13761 [==============================] - 60s 4ms/step - loss: 0.3888 - acc: 0.8191 - val_loss: 0.3965 - val_acc: 0.8489
Epoch 5/20
13761/13761 [==============================] - 59s 4ms/step - loss: 0.3815 - acc: 0.8191 - val_loss: 0.3946 - val_acc: 0.8489
Epoch 6/20
13761/13761 [==============================] - 59s 4ms/step - loss: 0.3773 - acc: 0.8191 - val_loss: 0.3942 - val_acc: 0.8489
Epoch 00006: early stopping
4832/4832 [==============================] - 73s 15ms/step
[0.45059645363431894, 0.8091887417218543]


In [0]:
model.save("ensamble_disgust.h5")
files.download("ensamble_disgust.h5")

In [0]:
Test_labels_surprise = pd.read_csv("mosi2uni_Test_labels_surprise.csv",header=None)
Train_labels_surprise = pd.read_csv("mosi2uni_Train_labels_surprise.csv",header=None)

In [0]:
model_audio_surprise = load_model("unimodal baselines/weights/audio/weights_rnn_surprise.h5")
model_video_surprise = load_model("unimodal baselines/weights/video/weights_rnn_surprise.h5")
model_text_surprise = load_model("unimodal baselines/weights/text/weights_rnn_surprise.h5")
early_fusion_surprise = load_model("multimodal baselines/v2 - early fusion/weights/early_fusion_surprise.h5")


In [0]:

concat_test = np.concatenate((video_test,audio_test),axis=-1)
concat_test = np.concatenate((concat_test, text_test_emb),axis=-1)
concat_train = np.concatenate((video_train,audio_train),axis=-1)
concat_train = np.concatenate((concat_train, text_train_emb),axis=-1)


In [0]:


for layer in model_audio_surprise.layers[:-1]:   
    layer.trainable = False
for layer in model_text_surprise.layers[:-1]:   
    layer.trainable = False
for layer in model_video_surprise.layers[:-1]:   
    layer.trainable = False
for layer in early_fusion_surprise.layers[:-1]:   
    layer.trainable = False


m_ef = Sequential()
i=0
for layer in early_fusion_surprise.layers[:]:
    layer.name = str(i)
    i=i+1
    m_ef.add(layer)


In [37]:

i_a = Input(shape=(20,74))
i_t = Input(shape=(20,300))
i_v = Input(shape=(20,35))
i_ef = Input(shape=(20,409))
y = concatenate([model_audio_surprise(i_a),model_text_surprise(i_t),model_video_surprise(i_v),m_ef(i_ef)])
y = Dense(1,activation='sigmoid')(y)
model = Model([i_a,i_t,i_v,i_ef],y)
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           (None, 20, 74)       0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           (None, 20, 300)      0                                            
__________________________________________________________________________________________________
input_27 (InputLayer)           (None, 20, 35)       0                                            
__________________________________________________________________________________________________
input_28 (InputLayer)           (None, 20, 409)      0                                            
__________________________________________________________________________________________________
model_4 (M

In [38]:
model.compile('adam','binary_crossentropy',metrics=['accuracy'],)
es = EarlyStopping(monitor='val_acc',mode='max' ,patience=5, min_delta=0.0001,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,patience=2,min_lr=0.00001)
model.fit([audio_train,text_train_emb,video_train,concat_train],Train_labels_surprise,128,epochs=20,validation_split=0.1,callbacks=[es,reduce_lr])
print(model.evaluate([audio_test,text_test_emb,video_test,concat_test],Test_labels_surprise))

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 90s 7ms/step - loss: 0.6663 - acc: 0.8929 - val_loss: 0.5900 - val_acc: 0.9091
Epoch 2/20
13761/13761 [==============================] - 61s 4ms/step - loss: 0.5224 - acc: 0.8966 - val_loss: 0.4207 - val_acc: 0.9091
Epoch 3/20
13761/13761 [==============================] - 62s 4ms/step - loss: 0.3638 - acc: 0.8966 - val_loss: 0.3355 - val_acc: 0.9091
Epoch 4/20
13761/13761 [==============================] - 62s 4ms/step - loss: 0.3134 - acc: 0.8966 - val_loss: 0.3284 - val_acc: 0.9091
Epoch 5/20
13761/13761 [==============================] - 62s 4ms/step - loss: 0.3035 - acc: 0.8966 - val_loss: 0.3228 - val_acc: 0.9091
Epoch 6/20
13761/13761 [==============================] - 62s 4ms/step - loss: 0.2984 - acc: 0.8966 - val_loss: 0.3218 - val_acc: 0.9091
Epoch 00006: early stopping
4832/4832 [==============================] - 76s 16ms/step
[0.33622099814430767, 0.9008692052980133]


In [0]:
model.save("ensamble_surprise.h5")
files.download("ensamble_surprise.h5")